In [2]:
import numpy as np
import pandas as pd  
# 1.读取数据
data_file_csv1 = 'inner_20_0_new.csv'
data_file_csv2 = 'ball_20_0_new.csv'
data_file_csv3 = 'outer_20_0_new.csv'
data_file_csv4 = 'comb_20_0_new.csv'
data_file_csv5 = 'health_20_0_new.csv'
origin_data1 = pd.read_csv(data_file_csv1)
origin_data2 = pd.read_csv(data_file_csv2)
origin_data3 = pd.read_csv(data_file_csv3)
origin_data4 = pd.read_csv(data_file_csv4)
origin_data5 = pd.read_csv(data_file_csv5)

print(origin_data1.shape)
print(origin_data2.shape)
print(origin_data3.shape)
print(origin_data4.shape)
print(origin_data5.shape)
origin_data1.head()

(1048560, 8)
(1048560, 8)
(1048560, 8)
(1048560, 8)
(1048560, 8)


,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,-0.117462,-0.001712,0.001460,0.007768,0.002619,0.003544,0.003097,-0.007119
1,-0.122317,-0.000914,-0.004050,-0.006717,0.003122,-0.012211,0.005323,-0.002731
2,-0.121311,0.004437,0.003101,0.007753,0.006050,0.003830,-0.007553,0.008774
3,-0.120751,-0.007136,-0.002774,0.002573,0.006198,0.001988,-0.001854,0.002958
4,-0.117236,-0.000008,0.000765,-0.007696,0.002301,0.004705,0.000281,0.005544


In [3]:
# 采用驱动端数据
data_column = 'Column3'
columns_name = ['inner','ball','outer','comb','health']
file_names = ['inner_20_0_new.csv', 'ball_20_0_new.csv', 'outer_20_0_new.csv', 'comb_20_0_new.csv', 'health_20_0_new.csv']
combine_data = pd.DataFrame()
for index in range(5):
    # 读取 CSV 文件
    data = pd.read_csv(file_names[index])
    dataList = data[data_column]
    combine_data[columns_name[index]] = dataList
print(combine_data.shape)
combine_data

(1048560, 5)


,inner,ball,outer,comb,health
0,0.001460,-0.007886,-0.000808,-0.392166,-0.458107
1,-0.004050,0.003791,0.002338,-0.379996,-0.449544
2,0.003101,-0.002126,-0.000834,-0.389422,-0.454772
3,-0.002774,-0.001577,0.001218,-0.388949,-0.458454
4,0.000765,0.002419,0.000473,-0.386126,-0.450946
...,...,...,...,...,...
1048555,-0.001255,-0.005304,0.003202,0.003767,-0.002427
1048556,-0.003281,0.002497,-0.001090,0.000229,0.000675
1048557,0.003114,-0.001674,0.003688,0.000527,0.004184
1048558,-0.007981,0.008260,-0.004952,0.000223,0.000635


In [4]:
combine_data.set_index('inner',inplace=True)
combine_data.to_csv('combine_data.csv')
combine_data.shape

(1048560, 4)

对于上述表格数据，要重新制作数据集，制作思路：
1. 对于上述表格每一列 数据，重新采样 信号长度为1024，滑动窗重叠率为0.5
2. 然后把重新采样的信号数据 放在另外的表格中，一行对应一个信号，而且在末尾添加分类标签

In [5]:
import numpy as np
import pandas as pd
import sklearn
from joblib import dump, load

# 时间步长 1024 和 重叠率 -0.5 
# window = 1024  step = 512  

def split_data_with_overlap(data, time_steps, lable, overlap_ratio=0.5):
    """
        data:要切分的时间序列数据,可以是一个一维数组或列表。
        time_steps:切分的时间步长,表示每个样本包含的连续时间步数。
        lable: 表示切分数据对应 类别标签
        overlap_ratio:前后帧切分时的重叠率,取值范围为 0 到 1,表示重叠的比例。
    """
    stride = int(time_steps * (1 - overlap_ratio))  # 计算步幅
    samples = (len(data) - time_steps) // stride + 1  # 计算样本数
    # 用于存储生成的数据
    Clasiffy_dataFrame = pd.DataFrame(columns=[x for x in range(time_steps + 1)])  
    data_list = []
    for i in range(samples):
        start_idx = i * stride
        end_idx = start_idx + time_steps
        temp_data = data[start_idx:end_idx].tolist()
        temp_data.append(lable)  # 对应哪一类
        data_list.append(temp_data)
    Clasiffy_dataFrame = pd.DataFrame(data_list, columns=Clasiffy_dataFrame.columns)
    return Clasiffy_dataFrame

# 归一化数据
def normalize(data):
    ''' (0,1)归一化
        参数:一维时间序列数据
    '''
    s= (data-min(data)) / (max(data)-min(data))
    return  s

# 数据集的制作
def make_datasets(data_file_csv, split_rate = [0.7,0.2,0.1]):
    '''
        参数:
        data_file_csv: 故障分类的数据集,csv格式,数据形状: 119808行  10列
        label_list: 故障分类标签
        split_rate: 训练集、验证集、测试集划分比例

        返回:
        train_set: 训练集数据
        val_set: 验证集数据
        test_set: 测试集数据
    '''
    # 1.读取数据
    origin_data = pd.read_csv(data_file_csv)
    # 2.分割样本点
    time_steps = 1024  # 时间步长
    overlap_ratio = 0.5  # 重叠率
    # 用于存储生成的数据# 10个样本集合
    samples_data = pd.DataFrame(columns=[x for x in range(time_steps + 1)])  
    # 记录类别标签
    label = 0
    # 使用iteritems()方法遍历每一列
    for column_name, column_data in origin_data.items():
        # 对数据集的每一维进行归一化
        # column_data = normalize(column_data)
        # 划分样本点  window = 1024  overlap_ratio = 0.5 
        split_data = split_data_with_overlap(column_data, time_steps, label, overlap_ratio)
        label += 1 # 类别标签递增
        samples_data = pd.concat([samples_data, split_data])
    # 随机打乱样本点顺序 
    # 打乱索引并重置索引
    samples_data = samples_data.sample(frac=1).reset_index(drop=True)
    # 3.分割训练集-、验证集、测试集
    sample_len = len(samples_data) # 每一类样本数量
    train_len = int(sample_len*split_rate[0])  # 向下取整
    val_len = int(sample_len*split_rate[1]) 
    train_set = samples_data.iloc[0:train_len,:]   
    val_set = samples_data.iloc[train_len:train_len+val_len,:]   
    test_set = samples_data.iloc[train_len+val_len:,:]   
    return  train_set, val_set, test_set, samples_data

# 生成数据集
train_set, val_set, test_set, samples_data = make_datasets('combine_data.csv')

# 保存数据
dump(train_set, 'train_set') 
dump(val_set, 'val_set') 
dump(test_set, 'test_set') 

['test_set']

In [6]:
print(type(samples_data))
samples_data.set_index(0, inplace=True)
samples_data.to_csv('samples_data.csv')
samples_data.shape

<class 'pandas.core.frame.DataFrame'>


(10230, 1024)

In [6]:
samples_data.shape

(10230, 1025)

In [7]:
# 制作数据集和标签
import torch

# 这些转换是为了将数据和标签从Pandas数据结构转换为PyTorch可以处理的张量，
# 以便在神经网络中进行训练和预测。

def make_data_labels(dataframe):
    '''
        参数 dataframe: 数据框
        返回 x_data: 数据集     torch.tensor
            y_label: 对应标签值  torch.tensor
    '''
    # 信号值
    x_data = dataframe.iloc[:,0:-1]
    # 标签值
    y_label = dataframe.iloc[:,-1]
    x_data = torch.tensor(x_data.values).float()
    y_label = torch.tensor(y_label.values.astype('int64')) # 指定了这些张量的数据类型为64位整数，通常用于分类任务的类别标签
    return x_data, y_label

# 加载数据
train_set = load('train_set') 
val_set = load('val_set') 
test_set = load('test_set') 

# 制作标签
train_xdata, train_ylabel = make_data_labels(train_set)
val_xdata, val_ylabel = make_data_labels(val_set)
test_xdata, test_ylabel = make_data_labels(test_set)
# 保存数据
dump(train_xdata, 'train_xdata')
dump(val_xdata, 'val_xdata')
dump(test_xdata, 'test_xdata')
dump(train_ylabel, 'train_ylabel')
dump(val_ylabel, 'val_ylabel')
dump(test_ylabel, 'test_ylabel')

['test_ylabel']

In [8]:
print('数据 形状：')
print(train_xdata.size(), train_ylabel.size())
print(val_xdata.size(), val_ylabel.size())
print(test_xdata.size(), test_ylabel.size())

数据 形状：
torch.Size([7161, 1024]) torch.Size([7161])
torch.Size([2046, 1024]) torch.Size([2046])
torch.Size([1023, 1024]) torch.Size([1023])
